In [28]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from pathlib import Path
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [29]:
# Mac OS, runs faster on CPU. Possibly beacuse of the small batch size and the subsequent non parallelization of the GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [30]:
def file_iterator():
    data_dir = Path('dependency_treebank')
    for data_file in filter(lambda f: os.path.isfile(data_dir/f) and f.endswith('.dp'), os.listdir(data_dir)):
        yield data_dir/data_file

In [31]:
train_set = []
test_set = []
val_set = []

train_split = 100
val_split = 150

file_counter = 0
for file in file_iterator():
    file_counter += 1
    if file_counter <= train_split:
        train_set.append(pd.read_csv(file, sep="\t", names=['token', 'pos'], usecols=[0, 1], engine='python'))
    elif file_counter <= val_split:
        val_set.append(pd.read_csv(file, sep="\t", names=['token', 'pos'], usecols=[0, 1], engine='python'))
    else:
        test_set.append(pd.read_csv(file, sep="\t", names=['token', 'pos'], usecols=[0, 1], engine='python'))

assert len(train_set) == 100
assert len(val_set) == 50
assert len(test_set) == 49

In [32]:
train_frame = pd.concat(train_set)
test_frame = pd.concat(test_set)
val_frame = pd.concat(val_set)

assert sum([e.shape[0] for e in train_set]) == train_frame.shape[0]
assert sum([e.shape[0] for e in test_set]) == test_frame.shape[0]
assert sum([e.shape[0] for e in val_set]) == val_frame.shape[0]

In [33]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Tokenizer.from_pretrained('bert-base-uncased')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_frame['token'].values)
vocab_size = len(tokenizer.word_index) + 1

encoded_doc = tokenizer.texts_to_sequences(train_frame['token'].values)

# Not necessary since we encode words and not sentences 
max_length = 1
padded_docs = pad_sequences(encoded_doc, maxlen=max_length, padding='post')

In [34]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [35]:
# create a weight matrix for words in training docs
# use 0 embedding as placeholder
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
	else:
		print("Not in Vocab", word)
	#TODO: Add edgecase OOV

Not in Vocab ratners
Not in Vocab crocidolite
Not in Vocab weisfield
Not in Vocab veraldi
Not in Vocab moleculon
Not in Vocab nipponese
Not in Vocab nagymaros
Not in Vocab ntg
Not in Vocab severable
Not in Vocab synergistics
Not in Vocab kalipharma
Not in Vocab tiphook
Not in Vocab erbamont
Not in Vocab nekoosa
Not in Vocab hallwood
Not in Vocab vitulli
Not in Vocab unicorp
Not in Vocab besuboru
Not in Vocab pramual
Not in Vocab twindam
Not in Vocab sidak
Not in Vocab micronite
Not in Vocab amphobiles
Not in Vocab pathlogy
Not in Vocab colorliner
Not in Vocab potables
Not in Vocab scypher
Not in Vocab mutchin
Not in Vocab akerfeldt
Not in Vocab gingl
Not in Vocab hasbrouk
Not in Vocab txo
Not in Vocab chemplus
Not in Vocab polyproplene
Not in Vocab bumkins
Not in Vocab purepac
Not in Vocab chinchon
Not in Vocab unenticing
Not in Vocab derchin
Not in Vocab phacoflex
Not in Vocab foldability
Not in Vocab satrum
Not in Vocab waymar
Not in Vocab purhasing
Not in Vocab ednie
Not in Vocab 29

In [9]:
# One hot encode train_frame['pos'].values
encoder = LabelEncoder()
encoder.fit(train_frame['pos'].values)
encoded_Y = encoder.transform(train_frame['pos'].values)
# convert integers to dummy variables (i.e. one hot encoded)
labels = np_utils.to_categorical(encoded_Y)



In [10]:
from tensorflow import keras
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1, trainable=False)
model.add(e)
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128), input_shape=(None, 50)))
model.add(Dense(44, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 100)            632700    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense (Dense)               (None, 44)                11308     
                                                                 
Total params: 878,504
Trainable params: 245,804
Non-trainable params: 632,700
_________________________________________________________________
None
Epoch 1/50


2022-12-06 02:20:22.493942: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1413/1413 [==============================] - 4s 2ms/step - loss: 0.9973 - accuracy: 0.7230
Epoch 2/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.5854 - accuracy: 0.8113
Epoch 3/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.5102 - accuracy: 0.8298
Epoch 4/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.4713 - accuracy: 0.8374
Epoch 5/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.4448 - accuracy: 0.8421
Epoch 6/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.4298 - accuracy: 0.8459
Epoch 7/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.4168 - accuracy: 0.8462
Epoch 8/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.4069 - accuracy: 0.8508
Epoch 9/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.4000 - accuracy: 0.8510
Epoch 10/50
1413/1413 [==============================] - 2s 2ms/step - loss: 0.3924 - accurac

In [11]:
# Using GRU instead of LSTM
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1, trainable=False)
model.add(e)
# A
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=128), input_shape=(None, 50)))
model.add(Dense(44, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 100)            632700    
                                                                 
 bidirectional_2 (Bidirectio  (None, 512)              549888    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 44)                22572     
                                                                 
Total params: 1,205,160
Trainable params: 572,460
Non-trainable params: 632,700
_________________________________________________________________
None
Epoch 1/50
1413/1413 [==============================] - 5s 2ms/step - loss: 0.8852 - accuracy: 0.7417
Epoch 2/50
1413/1413 [==============================] - 3s 2ms/step - loss: 0.5811 - accuracy: 0.8107
Epoch 3/50
1413/1413 [=======

In [19]:
# Use additional LSTM layer
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1, trainable=False)
model.add(e)
# A
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128, return_sequences=True), input_shape=(None, 50)))
model.add(keras.layers.LSTM(128))
model.add(Dense(44, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 1, 100)            632700    
                                                                 
 bidirectional_10 (Bidirecti  (None, 1, 256)           234496    
 onal)                                                           
                                                                 
 lstm_14 (LSTM)              (None, 128)               197120    
                                                                 
 dense_3 (Dense)             (None, 44)                5676      
                                                                 
Total params: 1,069,992
Trainable params: 437,292
Non-trainable params: 632,700
_________________________________________________________________
None
Epoch 1/50
1413/1413 [==============================] - 6s 3ms/step - loss: 0.9629 - accuracy: 0.7225

In [23]:
# Use additional Dense layer
from tensorflow import keras
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1, trainable=False)
model.add(e)
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128), input_shape=(None, 50)))
model.add(Dense(128))
model.add(Dense(44, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 1, 100)            632700    
                                                                 
 bidirectional_13 (Bidirecti  (None, 256)              234496    
 onal)                                                           
                                                                 
 dense_23 (Dense)            (None, 128)               32896     
                                                                 
 dense_24 (Dense)            (None, 44)                5676      
                                                                 
Total params: 905,768
Trainable params: 273,068
Non-trainable params: 632,700
_________________________________________________________________
None
Epoch 1/50
1413/1413 [==============================] - 4s 2ms/step - loss: 0.8624 - accuracy: 0.7429
